In [ ]:
#노선별 데이터 결합
from openpyxl import load_workbook
import pandas as pd
import glob

# Path to the directory with your files
file_path = "/content/*.xlsx"  # Update this path
all_files = sorted(glob.glob(file_path))  # Sort files alphabetically, ensuring Jan-Dec order

# List to collect data from each file
monthly_data_frames = []
for file in all_files:
    # Load workbook and select the active sheet
    wb = load_workbook(file)
    sheet = wb.active

    # Read data from each row
    data = []
    for row in sheet.iter_rows(values_only=True):
        # Check if any cell in the row is empty and replace it with the previous row's value
        if data and row[0] is None:
            # If current row's first cell is empty, use values from the last row in `data`
            last_row = data[-1]
            row = tuple(last_row[i] if cell is None else cell for i, cell in enumerate(row))
        data.append(row)

    # Convert data to a DataFrame and append to the list
    columns = data[0]  # First row as header
    df = pd.DataFrame(data[1:], columns=columns)
    monthly_data_frames.append(df)

# Concatenate all monthly DataFrames into one
combined_data = pd.concat(monthly_data_frames, ignore_index=True)

# Save the combined data to a new Excel file
combined_data.to_excel("노선별 이용량 23-combine.xlsx", index=False)

In [ ]:
#노선별 이용량 전처리
import pandas as pd
import numpy as np

# Replace 'your_file.csv' with the path to your CSV file
file_path = "/content/노선별 이용량 23-combine.csv"

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(file_path, encoding='utf-8-sig')

# Replace this list with the column names you want to check
columns_to_check = ["노선", "기종점", "월", "정류장순번", "정류장", "이용자유형"]

# Extract rows where any of the specified columns have null values
rows_with_nulls_in_columns = df[df[columns_to_check].isnull().any(axis=1)]

index_numbers_with_nulls = rows_with_nulls_in_columns.index

# Print the index numbers
print(f"Index numbers of rows with null values in columns {columns_to_check}:")
print(index_numbers_with_nulls)

df['노선'] = df['노선'].replace('합계', np.nan)

df.loc[index_numbers_with_nulls, '이용자유형'] = '합계'

df[columns_to_check] = df[columns_to_check].fillna(method='bfill')

df.fillna(0)

df.to_csv('노선별 이용량 전처리 23-combine.csv', index=False, encoding='utf-8-sig')